In [62]:
track_progress = True
# If true, the model prints statements as it completes certain long calculations

# Run necessary background code and generate baseline

In [65]:
# Background functions and calculations of adjustment factors
execfile('import_packages.py')
execfile('read_in_data.py')
execfile('amt_model.py')
execfile('ftc_model.py')
execfile('sec199_model.py')
execfile('ccr_model.py')
#execfile('interest_model.py')
execfile('usercode_taxcalc.py')

All packages successfully imported
Data imports complete
Beginning CCR model
Corporate depreciation adjustment calculated
Noncorporate depreciation adjustment calculated


In [66]:
execfile('gen_baseline.py')

Taxable income calculated
Investment matrices constructed
Corporate depreciation calculated
Noncorporate depreciation calculated
Capital paths calculated
Corporate net interest deduction calculated
Noncorporate interest deduction calculated
You loaded data for 2009.
Tax-Calculator startup automatically extrapolated your data to 2013.
Noncorporate tax rates calculated
Baseline complete


# Specify tax reform

In [67]:
btax_dict1 = {
    2017: {
        'tau_c': 0.357,
        #'tau_nc': 0.0,
        #'tau_amt': 0.2,
        #'depr_3yr_method': 'GDS',
        #'depr_3yr_bonus': 0.5,
        #'depr_5yr_method': 'GDS',
        #'depr_5yr_bonus': 0.5,
        #'depr_7yr_method': 'GDS',
        #'depr_7yr_bonus': 0.5,
        #'depr_10yr_method': 'GDS',
        #'depr_10yr_bonus': 0.5,
        #'depr_15yr_method': 'GDS',
        #'depr_15yr_bonus': 0.5,
        #'depr_20yr_method': 'GDS',
        #'depr_20yr_bonus': 0.5,
        #'depr_25yr_method': 'GDS',
        #'depr_25yr_bonus': 0.0,
        #'depr_275yr_method': 'GDS',
        #'depr_275yr_bonus': 0.0,
        #'depr_39yr_method': 'GDS',
        #'depr_39yr_bonus': 0.0,
        #'depr_land_bonus': 0.0,
        #'inventory_method': 'Mix'
    }
}
btax_dict2 = {
    'undepBasis_corp_hc': {0: 0.0},
    'undepBasis_noncorp_hc': {0: 0.0},
    'amt_repeal': {9e99: False},
    'pymtc_repeal': {9e99: False},
    'ftc_hc': {9e99: 0.0},
    'sec199_hc': {9e99: 0.0},
    'oldIntPaid_corp_hc': {0: 0.0},
    'newIntPaid_corp_hc': {0: 0.0},
    'netIntPaid_corp_hc': {0: 0.0},
    'oldIntPaid_noncorp_hc': {0: 0.0},
    'newIntPaid_noncorp_hc': {0: 0.0}
}
iit_params_ref = {}
execfile('reform_implementation.py')

You loaded data for 2009.
Tax-Calculator startup automatically extrapolated your data to 2013.
All parameters updated


# Calculate the investment response

In [75]:
# Set all elasticities
#     inv_usercost_c: elasticity of corporate investment w.r.t. the user cost of capital
#     inv_usercost_nc: elasticity of noncorporate investment w.r.t. the user cost of capital
#     inv_eatr: semi-elasticity of investment w.r.t. the effective average tax rate
#     debt_taxshield_c: semi-elasticity of the corporate debt/asset ratio w.r.t. the marginal tax shield, tau * (1 - hc)
#     debt_taxshield_nc: semi-elasticity of the noncorporate debt/asset ratio w.r.t. the marginal tax shield, tau * (1 - hc)
#     legalform_ratediff: semi-elasticity of the corporate share of business activity w.r.t. the (Tc - Tnc) tax rate differential


elast_dict = {
    'inv_usercost_c': 0.0,
    'inv_usercost_nc': 0.0,
    'inv_eatr_c': 0.0,
    'inv_eatr_nc': 0.0,
    'mne_share_c': 0.0,
    'mne_share_nc': 0.0,
    'debt_taxshield_c': 0.0,
    'debt_taxshield_nc': 0.0,
    'legalform_ratediff': -1.0
}
execfile('responses.py')

In [69]:
execfile('mini_combined.py')
response_results = inv_response(2017)

In [70]:
response_results

,Asset,deltaIc2014,deltaInc2014,deltaEc2014,deltaEnc2014,deltaIc2015,deltaInc2015,deltaEc2015,deltaEnc2015,deltaIc2016,...,deltaEc2025,deltaEnc2025,deltaIc2026,deltaInc2026,deltaEc2026,deltaEnc2026,deltaIc2027,deltaInc2027,deltaEc2027,deltaEnc2027
0,Aerospace products and parts manufacturing,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.300354,0.298111,0.0,0.0,0.300354,0.297631,0.0,0.0,0.300354,0.299022
1,Air transportation,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.139446,0.126643,0.0,0.0,0.139446,0.126618,0.0,0.0,0.139446,0.126604
2,Aircraft,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.179538,0.173637,0.0,0.0,0.179538,0.173737,0.0,0.0,0.179538,0.173830
3,"All other nonmanufacturing, n.e.c.",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.240354,0.238231,0.0,0.0,0.240354,0.237895,0.0,0.0,0.240354,0.239022
4,Amusement and recreation,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.121538,0.116298,0.0,0.0,0.121538,0.116359,0.0,0.0,0.121538,0.116407
5,Autos,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.448545,0.436229,0.0,0.0,0.448545,0.436186,0.0,0.0,0.448545,0.436182
6,Books,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.237326,0.224333,0.0,0.0,0.237326,0.224283,0.0,0.0,0.237326,0.224252
7,"Chemical manufacturing, ex. pharma and med",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.240354,0.238231,0.0,0.0,0.240354,0.237895,0.0,0.0,0.240354,0.239022
8,Communication,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.124725,0.116436,0.0,0.0,0.124725,0.116442,0.0,0.0,0.124725,0.116447
9,Communications,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.214061,0.205874,0.0,0.0,0.214061,0.205926,0.0,0.0,0.214061,0.205976


In [76]:
legal_response(2017)

You loaded data for 2009.
Tax-Calculator startup automatically extrapolated your data to 2013.
You loaded data for 2009.
Tax-Calculator startup automatically extrapolated your data to 2013.


# Run other responses and recalculate corporate tax revenue

In [81]:
execfile('gen_reform.py')

New investment matrices constructed
New corporate depreciation calculated
New noncorporate depreciation calculated
New capital paths calculated
New earnings calculated
New corporate net interest deduction calculated
New noncorporate interest deduction calculated
New taxable income calculated
Reform corporate tax revenue complete


### Check capital results

In [82]:
capPath_ref_corp

,FixedInv,FixedK,Investment,Kstock,TrueDep,taxDep,year
0,1027.370558,6192.813634,1105.741653,8525.386334,477.426504,884.575577,2014
1,985.082931,6762.771552,1092.837295,9180.871516,546.326287,905.241147,2015
2,887.419889,7287.566860,915.439816,9843.598272,600.386409,830.188301,2016
3,905.768870,7583.987473,934.368159,10171.241204,626.507756,853.410151,2017
4,943.650153,8015.027197,973.445528,10681.372140,662.115638,808.187354,2018
5,977.830553,8461.978637,1008.705160,11211.874626,698.146615,786.818998,2019
6,1011.551274,8918.815076,1043.490600,11755.938851,733.982585,625.772844,2020
7,1049.653015,9383.666212,1082.795388,12311.157188,769.619211,705.292789,2021
8,1091.136679,9857.997804,1125.588882,12878.157474,805.601248,773.311582,2022
9,1134.463748,10346.475651,1170.283986,13462.201368,842.529950,830.838028,2023


In [83]:
capPath_base_corp

,FixedInv,FixedK,Investment,Kstock,TrueDep,taxDep,year
0,1027.370558,6192.813634,1105.741653,8525.386334,477.426504,884.575577,2014
1,985.082931,6762.771552,1092.837295,9180.871516,546.326287,905.241147,2015
2,887.419889,7287.566860,915.439816,9843.598272,600.386409,830.188301,2016
3,920.096877,7703.955635,949.148567,10332.136132,636.418240,866.909916,2017
4,958.577389,8141.813810,988.844086,10850.336631,672.589389,820.971756,2018
5,993.298476,8595.835401,1024.661477,11389.230930,709.190326,799.265383,2019
6,1027.552612,9059.898359,1059.997174,11941.901497,745.593171,635.671702,2020
7,1066.257070,9532.102796,1099.923709,12505.902618,781.793520,716.449543,2021
8,1108.396947,10013.937655,1143.394137,13081.872062,818.344742,785.544298,2022
9,1152.409391,10510.142544,1188.796255,13675.154721,855.857605,843.980733,2023


In [85]:
capPath_ref_noncorp

,FixedInv,FixedK,Investment,Kstock,TrueDep,taxDep,year
0,271.318833,2566.871440,306.337312,6737.647577,159.355609,301.392835,2014
1,287.158825,2686.785985,338.014040,6905.064249,170.218220,328.459158,2015
2,292.681264,2837.223493,306.828509,7157.361498,182.182362,318.973624,2016
3,309.988148,3062.572645,324.971950,7565.731920,197.216674,343.280802,2017
4,322.952546,3236.635494,338.563005,7841.989026,208.425582,329.783711,2018
5,334.650364,3417.977967,350.826256,8131.074780,219.799436,323.954063,2019
6,346.190862,3604.422768,362.924584,8429.535650,231.150404,266.526293,2020
7,359.230710,3795.209323,376.594734,8735.659128,242.474046,296.971626,2021
8,373.427978,3990.619746,391.478252,9048.115091,253.928218,323.353623,2022
9,388.256130,4192.350970,407.023147,9369.443404,265.694662,345.975852,2023


In [84]:
capPath_base_noncorp

,FixedInv,FixedK,Investment,Kstock,TrueDep,taxDep,year
0,271.318833,2566.871440,306.337312,6737.647577,159.355609,301.392835,2014
1,287.158825,2686.785985,338.014040,6905.064249,170.218220,328.459158,2015
2,292.681264,2837.223493,306.828509,7157.361498,182.182362,318.973624,2016
3,303.458510,2998.062147,318.126691,7406.366186,193.062471,336.049883,2017
4,316.149825,3168.458510,331.431462,7676.804170,204.035274,322.837096,2018
5,327.601237,3345.981157,343.436398,7959.800578,215.169547,317.130245,2019
6,338.898645,3528.498656,355.279885,8251.974623,226.281416,260.912143,2020
7,351.663819,3715.266454,368.662085,8551.649869,237.366536,290.716170,2021
8,365.562034,3906.560722,383.232094,8857.524212,248.579435,316.542453,2022
9,380.077843,4104.042649,398.449550,9172.084016,260.098029,338.688164,2023


### Check combined results

In [86]:
combined_ref

,year,ebitda,taxDep,nid,sec199,taxinc,gbc_adj,tau,taxbc,ftc,amt,pymtc,taxrev
0,2014,2445.993351,884.575577,231.441998,34.306208,1295.669568,0.021643,0.347,421.555664,101.582376,4.457941,3.700228,320.731000
1,2015,2520.645171,905.241147,243.981131,34.855388,1336.567506,0.021643,0.347,434.862110,91.794692,4.529305,3.799722,343.797000
2,2016,2327.856171,830.188301,248.001094,34.816996,1214.849781,0.021643,0.347,395.260349,96.318143,4.524316,3.895522,299.571000
3,2017,2390.549964,853.410151,263.907081,34.407918,1238.824814,0.021643,0.357,415.449051,98.379715,4.345916,3.916140,317.499111
4,2018,2405.767613,808.187354,290.869045,34.473648,1272.237566,0.021643,0.357,426.654264,95.095766,4.354218,3.972573,331.940143
5,2019,2480.989505,786.818998,323.744400,34.799011,1335.627095,0.021643,0.357,447.912411,96.460224,4.395313,4.022686,351.824814
6,2020,2480.558236,625.772844,356.760597,35.298558,1462.726237,0.021643,0.357,490.536047,101.941181,4.458408,4.071615,388.981659
7,2021,2614.955495,705.292789,387.537625,36.029802,1486.095279,0.021643,0.357,498.373028,107.306507,4.550769,4.122404,391.494886
8,2022,2750.360211,773.311582,415.918677,37.019036,1524.110916,0.021643,0.357,511.121853,113.018728,4.675715,4.178652,398.600188
9,2023,2872.158143,830.838028,441.485469,38.366497,1561.468149,0.021643,0.357,523.649877,119.725385,4.845907,4.243920,404.526479


In [87]:
combined_base

,year,taxrev,amt,pymtc,ftc,taxbc,gbc_adj,tau,taxinc,sec199,taxDep,nid,ebitda
0,2014,320.731,4.457941,3.700228,101.582376,421.555664,0.021643,0.347,1295.669568,34.306208,884.575577,231.441998,2445.993351
1,2015,343.797,4.529305,3.799722,91.794692,434.862110,0.021643,0.347,1336.567506,34.855388,905.241147,243.981131,2520.645171
2,2016,299.571,4.524316,3.895522,96.318143,395.260349,0.021643,0.347,1214.849781,34.816996,830.188301,248.001094,2327.856171
3,2017,310.006,4.541886,3.978088,99.935946,409.378148,0.021643,0.347,1258.241448,34.952204,866.909916,268.261581,2428.365149
4,2018,324.343,4.550562,4.052119,96.600049,420.444606,0.021643,0.347,1292.254685,35.018974,820.971756,295.578104,2443.823519
5,2019,343.899,4.593510,4.117569,97.986091,441.409149,0.021643,0.347,1356.690115,35.349483,799.265383,328.930336,2520.235317
6,2020,380.350,4.659451,4.180064,103.553749,483.424362,0.021643,0.347,1485.825688,35.856932,635.671702,362.442904,2519.797226
7,2021,382.662,4.755976,4.243011,109.003947,491.152982,0.021643,0.347,1509.579935,36.599743,716.449543,393.691242,2656.320464
8,2022,389.490,4.886556,4.310368,114.806528,503.720339,0.021643,0.347,1548.206251,37.604625,785.544298,422.511919,2793.867094
9,2023,395.128,5.064423,4.386026,121.619274,516.068877,0.021643,0.347,1586.160018,38.973402,843.980733,448.477548,2917.591701


In [88]:
compare = np.asarray(combined_ref['taxrev'] - combined_base['taxrev'])

print "Total corporate tax revenue change, 2014-2027: " + str(np.round(sum(compare), 3))
print "Total corporate tax revenue change, 2017-2026: " + str(np.round(sum(compare[3:13]), 3))
print "Total corporate tax revenue change, 2018-2027: " + str(np.round(sum(compare[4:]), 3))

Total corporate tax revenue change, 2014-2027: 100.055
Total corporate tax revenue change, 2017-2026: 89.226
Total corporate tax revenue change, 2018-2027: 92.562


In [89]:
for i in compare:
    print i

5.68434188608e-14
-1.13686837722e-13
-1.13686837722e-13
7.49311148508
7.59714274149
7.92581402987
8.63165947991
8.83288600142
9.11018760267
9.39847864689
9.72103977438
10.0756993059
10.4402790102
10.8284058604


## Calculate percent changes in investment income and pass-through income

In [90]:
execfile('multipliers_for_taxcalc.py')

In [91]:
indiv_gfactors

,Investment,year,inc_aftertax_base,inc_aftertax_ref,equity,debt,SchC_pos,SchC_neg,e26270_pos,e26270_neg
0,1105.741653,2014,1416.393849,1416.393849,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
1,1092.837295,2015,1386.540753,1386.540753,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
2,915.439816,2016,1179.897668,1179.897668,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
3,949.148567,2017,1213.679328,1182.636016,0.974422,0.983768,1.021502,1.021581,1.021477,1.021534
4,988.844086,2018,1151.313026,1120.842787,0.973534,0.984068,1.021508,1.021554,1.021494,1.021527
5,1024.661477,2019,1138.215655,1107.273675,0.972815,0.984234,1.021512,1.021539,1.021504,1.021523
6,1059.997174,2020,1031.411151,1000.833394,0.970353,0.984322,1.021514,1.021530,1.021509,1.021521
7,1099.923709,2021,1098.173702,1066.303773,0.970979,0.984369,1.021516,1.021525,1.021513,1.021519
8,1143.394137,2022,1163.520433,1130.240098,0.971397,0.984395,1.021516,1.021522,1.021515,1.021518
9,1188.796255,2023,1218.128548,1183.616245,0.971668,0.984409,1.021517,1.021520,1.021516,1.021518


## Pass changes to Tax-Calculator

In [92]:
indiv_rev_impact = distribute_results(iit_params_ref)

You loaded data for 2009.
Tax-Calculator startup automatically extrapolated your data to 2013.
You loaded data for 2009.
Tax-Calculator startup automatically extrapolated your data to 2013.


In [93]:
for x in indiv_rev_impact:
    print x
print sum(indiv_rev_impact[3:])

2.35991903467e-14
7.53214734706e-14
0.0
-0.930664845856
-1.50791284047
-1.72694842591
-2.7458910182
-2.67039282027
-2.63803293153
-2.60263070099
-2.59654764931
-2.59789874622
-2.64928386632
-2.99263096745
-25.6588348125


In [94]:
ModelResults = pd.DataFrame({'year': range(2014,2028), 'IndivTaxRev': indiv_rev_impact})
ModelResults['CorpTaxRev'] = combined_ref['taxrev'] - combined_base['taxrev']
ModelResults['RevenueChange'] = ModelResults['IndivTaxRev'] + ModelResults['CorpTaxRev']
ModelResults

,IndivTaxRev,year,CorpTaxRev,RevenueChange
0,2.359919e-14,2014,5.684342e-14,8.044261e-14
1,7.532147e-14,2015,-1.136868e-13,-3.836536e-14
2,0.000000e+00,2016,-1.136868e-13,-1.136868e-13
3,-9.306648e-01,2017,7.493111e+00,6.562447e+00
4,-1.507913e+00,2018,7.597143e+00,6.089230e+00
5,-1.726948e+00,2019,7.925814e+00,6.198866e+00
6,-2.745891e+00,2020,8.631659e+00,5.885768e+00
7,-2.670393e+00,2021,8.832886e+00,6.162493e+00
8,-2.638033e+00,2022,9.110188e+00,6.472155e+00
9,-2.602631e+00,2023,9.398479e+00,6.795848e+00


In [95]:
print "Total tax revenue change, 2014-2027: " + str(np.round(sum(ModelResults['RevenueChange']), 3))
print "Total tax revenue change, 2017-2026: " + str(np.round(sum(ModelResults['RevenueChange'][3:13]), 3))
print "Total tax revenue change, 2018-2027: " + str(np.round(sum(ModelResults['RevenueChange'][4:]), 3))

Total tax revenue change, 2014-2027: 74.396
Total tax revenue change, 2017-2026: 66.56
Total tax revenue change, 2018-2027: 67.833


In [96]:
for i in range(3,14):
    print ModelResults['RevenueChange'][i]
print '\n'
print sum(ModelResults['RevenueChange'][3:13])

6.56244663922
6.08922990103
6.19886560396
5.88576846172
6.16249318116
6.47215467114
6.79584794589
7.12449212507
7.47780055965
7.79099514383
7.83577489296


66.5600942327
